In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# yfinanceを使用して日経225のデータを取得
NK225_data = yf.download('^N225', start='任意の日付', end=datetime.now().strftime('%Y-%m-%d'))

# 'Adj Close'列のデータを取得
NK225_prices = NK225_data['Adj Close']

# 欠損値を削除
NK225_prices = NK225_prices.dropna()

# データをCSVファイルに保存
NK225_prices.to_csv('NK225_prices.csv')

# CSVファイルからデータを読み込む
NK225_prices = pd.read_csv('NK225_prices.csv', index_col='Date', parse_dates=True)

# 特徴量エンジニアリング: # 適切な特徴量を選択
NK225_prices['適切な特徴量'] = NK225_prices['Adj Close'].rolling(window=5).mean()

# 欠損値を削除（再度確認）
NK225_prices = NK225_prices.dropna()

# 特徴量とターゲットを定義
X = NK225_prices[['Adj Close', '適切な特徴量']]  
y = NK225_prices['Adj Close']

# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレストモデルを構築
model = RandomForestRegressor()
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 予測結果の可視化
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='実際の株価')
plt.plot(y_test.index, y_pred, label='予測株価', linestyle='--')
plt.title('日経225 株価予測')
plt.xlabel('日付')
plt.ylabel('株価')
plt.legend()
plt.show()
